In [1]:
import pandas as pd
# 载入语料
raw = pd.read_csv("金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="utf-8" ,engine='python')

# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量
del raw['head']
del raw['mid']
del raw['len']

chap3 = raw[raw['chap'] == 3].copy()
chap3.reset_index(drop=True, inplace=True)
chap3['paraidx'] = chap3.index
del chap3['chap']
del chap3['paraidx']

chap4 = raw[raw['chap'] == 4].copy()
chap4.reset_index(drop=True, inplace=True)
chap4['paraidx'] = chap4.index
del chap4['chap']
del chap4['paraidx']

In [3]:
import jieba

cuttxt = lambda chap3: " ".join(jieba.lcut(chap3)) # 这里不做任何清理工作，以保留情感词
chap3["cut1"] = chap3.txt.apply(cuttxt)

In [4]:
import jieba

cuttxt = lambda chap4: " ".join(jieba.lcut(chap4)) # 这里不做任何清理工作，以保留情感词
chap4["cut2"] = chap4.txt.apply(cuttxt)

In [5]:
# 从原始语料df中提取出所需的前两章段落
raw12 = raw[raw.chap.isin([3,4])]
raw12ana = raw12.iloc[list(raw12.txt.apply(len) > 50), :] # 只使用超过50字的段落
raw12ana.reset_index(drop = True, inplace = True)
print(len(raw12ana))
raw12ana.head()

338


,txt,chap
0,忽听得巨钟下的铜缸内当当当响声不绝，不知里面是何怪物，众僧面面相觑，手足无措，齐声口诵《高王...,3.0
1,柯镇恶双腿中剑，受伤不轻，神智却仍清明，从怀中摸出解毒药来，命僧人分别去给丘处机及韩小莹服下...,3.0
2,朱聪与南希仁所受内伤甚重。全金发腰间所受的这一脚也着实不轻。张阿生胳臂折断，胸口受震，一时痛...,3.0
3,过了数日，丘处机与韩小莹身上所中的毒都消解了。丘处机精通医道，开了药方给朱聪等人调治，又分别...,3.0
4,过了一会，韩小莹首先说道：“丘道长能干英明，天下皆知，我们七兄弟也不是初走江湖之人，这次大家...,3.0


In [6]:
# 分词和预处理
import jieba

cuttxt = lambda x: " ".join(jieba.lcut(x)) # 这里不做任何清理工作，以保留情感词
raw12ana["cleantxt"] = raw12ana.txt.apply(cuttxt) 
raw12ana.head()

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,txt,chap,cleantxt
0,忽听得巨钟下的铜缸内当当当响声不绝，不知里面是何怪物，众僧面面相觑，手足无措，齐声口诵《高王...,3.0,忽 听 得 巨钟 下 的 铜 缸内 当当 当 响声 不 绝 ， 不知 里面 是 何 怪物 ，...
1,柯镇恶双腿中剑，受伤不轻，神智却仍清明，从怀中摸出解毒药来，命僧人分别去给丘处机及韩小莹服下...,3.0,柯镇恶 双腿 中剑 ， 受伤 不 轻 ， 神智 却 仍 清明 ， 从 怀中 摸 出 解毒药 ...
2,朱聪与南希仁所受内伤甚重。全金发腰间所受的这一脚也着实不轻。张阿生胳臂折断，胸口受震，一时痛...,3.0,朱聪 与 南希仁 所 受 内伤 甚重 。 全金发 腰间 所受 的 这 一脚 也 着实 不 轻...
3,过了数日，丘处机与韩小莹身上所中的毒都消解了。丘处机精通医道，开了药方给朱聪等人调治，又分别...,3.0,过 了 数日 ， 丘处机 与 韩小莹 身上 所中 的 毒 都 消解 了 。 丘处机 精通 医...
4,过了一会，韩小莹首先说道：“丘道长能干英明，天下皆知，我们七兄弟也不是初走江湖之人，这次大家...,3.0,过 了 一会 ， 韩小莹 首先 说道 ： “ 丘道长 能干 英明 ， 天下 皆知 ， 我们 ...


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer() 

wordmtx = countvec.fit_transform(raw12ana.cleantxt)
# 作用：将数据集划分为 训练集和测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(wordmtx, raw12ana.chap, 
    test_size = 0.3, random_state = 111)

In [14]:
print(x_test)

  (0, 1490)	1
  (0, 2929)	1
  (0, 5021)	1
  (0, 4670)	1
  (0, 12)	2
  (0, 3348)	1
  (0, 802)	1
  (0, 1445)	1
  (0, 423)	1
  (0, 1783)	1
  (0, 4810)	1
  (0, 677)	1
  (0, 1305)	1
  (0, 559)	1
  (0, 3416)	1
  (0, 3778)	1
  (0, 5524)	1
  (0, 1406)	1
  (0, 2577)	1
  (0, 800)	1
  (0, 3364)	1
  (0, 1002)	1
  (0, 3217)	1
  (0, 4483)	1
  (0, 3576)	1
  :	:
  (101, 963)	1
  (101, 2247)	1
  (101, 717)	1
  (101, 1850)	1
  (101, 5036)	1
  (101, 487)	1
  (101, 5671)	1
  (101, 1988)	1
  (101, 5432)	1
  (101, 285)	1
  (101, 4)	1
  (101, 4824)	1
  (101, 4614)	1
  (101, 1411)	1
  (101, 169)	1
  (101, 4435)	1
  (101, 1729)	1
  (101, 5355)	1
  (101, 5329)	1
  (101, 5362)	1
  (101, 817)	1
  (101, 3624)	1
  (101, 4036)	1
  (101, 2095)	1
  (101, 3430)	1


In [16]:
m = x_test.todense()

In [17]:
m

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)